In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import openpyxl
import datetime
import re
import string
from bs4 import BeautifulSoup
import requests


In [3]:
URL ='/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Tilastopaja_Asian_Games_2018.html'  # open a locally saved copy of html
HTMLFileToBeOpened = open(URL, "r") 
contents = HTMLFileToBeOpened.read() 
soup = BeautifulSoup(contents, 'lxml') 


In [78]:
# Traverse html with find_all and enumerate
# <TABLE> then <TR> then <TD> then <A>

lists=[]
counts=[]
values=[]
names=[]
wind=''
stage=''
event=''
gender=''
heat=''
dict_result2=[]  # length 2 dict
row=[]

temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'DICT_RESULT'])


h1s = soup.find_all('h1')

for h1 in h1s:
    
    print(h1.text)


for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete
        
    for element in i.find_all('tr'): # <tr> separates each athlete/section
        
    #    print('NEW SECTION')
        
       # for td in element.find_all('td'):
    
        counts=[]
        values=[]
            
        for count, value in enumerate(element.find_all('td')):
            
            for n in value.find_all('a'):  # extract the name and put into a list
                                
                names.append(n.text)
                
                                   
                     
            if len(lists)==2:
                
                heat=''
                wind=''
                dict_result=''
                dict_result2=''
                stage=''
                
                if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                    gender = 'Male'
                    
                if 'Women' in lists[1]:
                    gender = 'Female'
                    
                     
            if 'Date' in value.text:
                date=value.text
                          
            if 'Wind' in value.text:
                wind=value.text
     #           print('WINDDDDD', value.text)
                
    #        print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB
            
            counts.append(count)
            values.append(value.text)
            
          
                    
        lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary
        
        
        if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it stage
            
            heat = lists[1]
            wind = lists[2]  # wind can also appear in list of length 5
            
            dict_result=[]
            
        if len(lists)==3 and 'Wind' not in lists[2]:
            
            if 'Semifinals' in lists[1]:
                
                stage = 'Semifinals'
                
            if 'Heats' in lists[1]:
                
                stage = 'Heats'
                              
            if 'Heat 1' in lists[1]:
                
                heat = 'Heat 1'
                
            if 'Heat 2' in lists[1]:
                
                heat = 'Heat 2'
            
            if 'Heat 3' in lists[1]:
                
                heat = 'Heat 3'
            
            if 'Heat 4' in lists[1]:
                
                heat = 'Heat 4'
            
            
            if len(lists[2])>1:
                                
                date = lists[2]  # set date only if lists[2] is not null
            
            dict_result = []
                
        if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered
            
            date = lists[2]
            event = lists[1]
            wind = ' '
            stage = ' '
            heat = ' '
            
            dict_result = []
            dict_result2 = []
            
            if 'Wind' in lists[4]:
                
                wind = lists[4][5:]  # slice out the 'Wind:'
            
                        
        if len(lists)==11: # dict length 10 contains athlete info
                        
            dict_result=lists
            
            dict_results2 = [] # reset attempts
            
            
            
        
                
        if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        
        
     #   print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)
        
        print('NAMES', names)
        try:
            
            if (dict_result[0] and dict_result[1] and event) is not None:
                        
                row.append(dict_result[0]) # rank
                row.append(dict_result[1]) # name  (dict_result[1])
               # row.append(names[1]) # name  (dict_result[1])

                row.append(dict_result[3]) # nationality
                row.append(dict_result[5]) # result incl. DNS
                row.append(dict_result[9]) # Q/q
                row.append('18th Asian Games')
                row.append('2018')
                row.append(date)
                row.append(event)
                row.append('Gelora Bung Karno Stadium')
                row.append(gender)
                row.append(stage)
                row.append(heat)
                row.append(wind)
                row.append(dict_result[4]) # DOB                    
                row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                row.append(dict_result2) # DICT_RESULT


            #print(row)
            
        except:
            
            pass
        
        if len(row)!=0:
        
            temp_df.loc[len(temp_df)] = row

      #  print('LENGTH', len(lists))

        
        dict_result2 = []
        row=[]
        
        
    wind=''
    heat=''
    date=''
    
    lists=[]
    dict_result = []
    row=[]
                        
                
                
            
            
            


        
      

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [79]:
temp_df

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Su Bingtian Su Bingtian ('89),CHN,9.92,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,29 Aug 89,« »,,[]
1,2,Tosin OgunodeTosin Ogunode ('94),QAT,10.00,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,2 Mar 94,PB »,,[]
2,3,Ryota YamagataRyota Yamagata ('92),JPN,10.00,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,10 Jun 92,=PB »,,[]
3,4,Abdullah Abkar MohammedAbdullah Abkar Mohammed...,KSA,10.10,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,1 Jun 97,« »,,[]
4,5,Yang Chun-Han Yang Chun-Han ('97),TPE,10.17,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,1 Jan 97,« »,,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533,8,Wang Qingling Wang Qingling ('93),CHN,39.33,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,14 Jan 93,« »,,"{0: ' ', 1: ' 35.60 38.21 39.33..."
1534,9,Jeong Yeon-Jin Jeong Yeon-Jin ('92),KOR,35.64,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,25 Mar 92,« »,,
1535,9,Jeong Yeon-Jin Jeong Yeon-Jin ('92),KOR,35.64,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,25 Mar 92,« »,,"{0: ' ', 1: ' 35.64 33.09 31.26..."
1536,,Norliana KamaruddinNorliana Kamaruddin ('91),MAS,NM,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,26 Mar 91,« »,,
